In [ ]:
!pip install markdown2 pdfkit

In [ ]:
import os
import re
import markdown2
import pdfkit

# Configuration
md_folder = '/home/md/'  # Path to your folder with .md files
output_pdf = '/home/md_output.pdf'  # Output PDF path
temp_html_path = '/tmp/temp_md.html'  # Temporary HTML file path

# Collect and naturally sort all .md files in the folder
md_files = [
    os.path.join(md_folder, f)
    for f in os.listdir(md_folder)
    if f.lower().endswith(".md")
]

def natural_sort_key(file):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', file)]

md_files.sort(key=natural_sort_key)

# Combine all markdown content into a single string (no file titles)
combined_markdown = ''
for md_file in md_files:
    with open(md_file, 'r', encoding='utf-8') as f:
        content = f.read()
        combined_markdown += content + "\n\n"

# Convert combined markdown to HTML using markdown2 with table support
html_content = markdown2.markdown(
    combined_markdown,
    extras=['fenced-code-blocks', 'tables']
)

# Add robust CSS to ensure tables render properly in PDF
table_css = """
<style>
    body {
        font-family: Arial, sans-serif;
        font-size: 12px;
        padding: 20px;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin: 10px 0;
    }
    th, td {
        border: 1px solid #444;
        padding: 6px 10px;
        word-wrap: break-word;
        max-width: 200px;
        text-align: left;
    }
    th {
        background-color: #f0f0f0;
    }
    pre, code {
        background-color: #f8f8f8;
        padding: 4px;
        border-radius: 4px;
        display: block;
        overflow-x: auto;
    }
</style>
"""

# Save full HTML content to a temporary HTML file
with open(temp_html_path, 'w', encoding='utf-8') as f:
    f.write(table_css + html_content)

# Optional: Pass wkhtmltopdf path manually if not in PATH
# config = pdfkit.configuration(wkhtmltopdf='/usr/bin/wkhtmltopdf')  # Adjust if needed

# Convert the HTML to PDF
pdfkit.from_file(temp_html_path, output_pdf)  # add configuration=config if needed

print(f"✅ PDF created successfully: {output_pdf}")